In [1]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp
pip install jupyterthemes

  Using cached pip-20.2.4-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=418427c05ad43e10c1b49f0c4935460ccfab2308ad5c9097cdede84eac259700
  Stored in directory: /home/ec2-user/.cache/pip/wheels/88/2a/d4/f2e9023989d4d4b3574f268657cb6cd23994665a038803f547
Successfully built smart-open


In [2]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [3]:
import time 
import re
import os
import trp
import random 
import boto3
import minecart
import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session
from io import BytesIO

%matplotlib inline

In [4]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="FOCUS-OCR/SubsetTest/"

# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [9]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2)
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # itterate through each row within the provided table
    for row in range(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in range(M)]
        
    return pd.DataFrame(arr)

In [10]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        # itterate through page tables
        for table in page.tables: 
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty (column 0 = line items)
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
            
            # make sure dataframe is not empty  
            if df.values.size > 0:
                colIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity
                assetCheck = df[colIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity
                debtCheck1 = df[colIndex].str.contains('Liabilities$|^Liabilities', 
                                                      regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[colIndex].str.contains('Liability$|^Liability', 
                                                      regex=True, flags=re.IGNORECASE)

                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty
                check2 = debtCheck1[debtCheck1 == True].empty
                check3 = debtCheck2[debtCheck2 == True].empty

                # if figure matched exactly, we assume this is balance sheet 
                if not check1:
                    catDF.append(df)

                    if check2 == False or check3 == False:  
                        return pd.concat(catDF)
                

## Extract Balance Sheet information

In [11]:
subsetFolder = np.array(session.list_s3_files(bucket, 'Input/SubSets/'))[1:]

In [12]:
subsetFolder

array(['Input/SubSets/1000146-02_subset.pdf',
       'Input/SubSets/1000146-03_subset.pdf',
       'Input/SubSets/1000146-04_subset.pdf', ...,
       'Input/SubSets/99947-17_subset.pdf',
       'Input/SubSets/99947-18_subset.pdf',
       'Input/SubSets/99947-19_subset.pdf'], dtype='<U35')

In [13]:
print('Total files needed to be converted {}'.format(subsetFolder.size))

Total files needed to be converted 83129


In [ ]:
# script to perform OCR (using Textract) for X-17A-5 subsets
outFolder = 'Output/BalanceSheet/'
csvDirectory = np.array(session.list_s3_files(bucket, outFolder))
errorLog = []

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(subsetFolder[2493:10000]):
    # baseFile name (CIK)-{Year}
    baseFile = key.split('/')[-1].split('_')[0]
    fileName = baseFile+'.csv'
    print('\nFile {}'.format(i))
    # if file is not found in directory 
    if outFolder+fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", key)
        
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            # checks for type of return 
            if type(tempDF) == pd.DataFrame:
                # writing data frame to .csv file
                tempDF.to_csv(fileName)

                # the folder used to export to
                xFolder = 'Output/BalanceSheet/'+fileName

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=xFolder, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                print('\tSaved {} file to s3 bucket'.format(baseFile+'.csv'))
            else:
                print('No Balance Sheet found in {}'.format(baseFile))
                errorLog.append(baseFile)
        else:
            print('{} could not be parsed'.format(baseFile))
            errorLog.append(baseFile)
    else:
        print('{} has been downloaded'.format(fileName))

print('==========================\nOCR is completed')


File 0
1013851-04.csv has been downloaded

File 1
1013851-05.csv has been downloaded

File 2
1013851-06.csv has been downloaded

File 3
1013853-02.csv has been downloaded

File 4
1014228-02.csv has been downloaded

File 5
1014228-03.csv has been downloaded

File 6
1014230-02.csv has been downloaded

File 7
1014230-03.csv has been downloaded

File 8
1014230-04.csv has been downloaded

File 9
1014230-05.csv has been downloaded

File 10
1014230-06.csv has been downloaded

File 11
1014230-07.csv has been downloaded

File 12
1014230-08.csv has been downloaded

File 13
1014230-09.csv has been downloaded

File 14
1014230-10.csv has been downloaded

File 15
1014639-02.csv has been downloaded

File 16
1014639-03.csv has been downloaded

File 17
1014639-04.csv has been downloaded

File 18
1014639-05.csv has been downloaded

File 19
1014639-06.csv has been downloaded

File 20
1014639-07.csv has been downloaded

File 21
1014639-08.csv has been downloaded

File 22
1014639-09.csv has been downloade

Started job with id: 92a902e8ab02291f2e96d5b8ca0f54335f411840a8740af3235d3b121760e3a0
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
No Balance Sheet found in 1015273-02

File 188
1015273-03.csv has been downloaded

File 189
1015273-05.csv has been downloaded

File 190
1015273-06.csv has been downloaded

File 191
Started job with id: b292b3deb7a465d8e6d93bf5c9e6ff3a615e8affe528122ad65ae84f65f21a0e
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
No Balance Sheet found in 1015273-07

File 192
1015273-08.csv has been downloaded

File 193
Started job with id: d924d923c4e9ddc10d309a288dbf16ec38d6cb61041b9e145f576d764d5f1ad8
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset p

Started job with id: 497c47650b0c104fdda8f436c3a9abca52c2b45d40546e1cbbe6077882e74089
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
No Balance Sheet found in 1016457-13

File 345
1016457-15.csv has been downloaded

File 346
1016457-16.csv has been downloaded

File 347
1016458-02.csv has been downloaded

File 348
1016458-03.csv has been downloaded

File 349
1016458-04.csv has been downloaded

File 350
Started job with id: a537ca60df777722060dfb26803f1c84a8ed67158f289c7fd0311d6eefd9a1ca
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
No Balance Sheet found in 1016458-05

File 351
1016460-02.csv has been downloaded

File 352
1016460-03.

Started job with id: b1793dbf97ef4211416aa0d89c098561f69d418cdcfdf4c1992d574ac2989c31
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
None
No Balance Sheet found in 1017599-10

File 461
1017599-11.csv has been downloaded

File 462
Started job with id: 4b64475a1bbdbf974adce900c90b3811b52bbcf1817d489e6c36e11465445182
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
No Balance Sheet found in 1017599-12

File 463
1017753-02.csv has been downloaded

File 464
1017753-03.csv has been downloaded

File 465
1017753-04.csv has been downloaded

File 466
1017753-05.csv has been downloaded

File 467
1017753-06.csv has been downloaded

File 468
1017753-07.csv has been downloaded

File 469
1017753-08.csv has been downloaded

File 470
1017760-02.csv has been downloaded

File 471
1017760-03.csv has been downloaded

File 472
St

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
None
No Balance Sheet found in 1019119-04

File 584
1019119-05.csv has been downloaded

File 585
Started job with id: 6973738db4846b8cbfa7061227fdeb2c1d940e66ea8f0a89bed9ccc50be33836
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None
No Balance Sheet found in 1019119-06

File 586
1019119-07.csv has been downloaded

File 587
1019119-08.csv has been downloaded

File 588
1019119-09.csv has been downloaded

File 589
1019119-10.csv has been downloaded

File 590
Started job with id: c393a0e7fa913618f4467990474722d9237c378245b10142f9aa8c5440f36d1e
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN

...

Started job with id: de2a1b22c0911d0adf6bb55606ea5d232a8726ee582bf35b674baf8e179a83f0
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                        0     

Resultset page recieved: 1
Resultset page recieved: 2
                                        0          1
0               Cash and Cash Equivalents   $ 47,110
1                     Accounts Receivable    408,149
2                            Prepaid Fees      1,356
3               Receivable from Affiliate        658
4             Investments in Partnerships      2,000
5                            Total Assets  $ 459,273
6        LIABILITIES AND MEMBERS' EQUITY:           
7                             LIABILITIES           
8   Accounts Payable and Accrued Expenses   $ 25,044
9                   Deferred Income Taxes     14,879
10                      Total Liabilities     39,923
11                        MEMBERS' EQUITY    419,350
12  Total Liabilities and Members' Equity  $ 459,273
	Saved 1022464-11.csv file to s3 bucket

File 1041
Started job with id: 1b17231459b03ce8994d6cebce2576bbc6204f98a14957229637c10d292d7695
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROG

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                        0          1
0                                    Cash   $ 80,915
1                     Accounts Receivable     21,154
2              Due from Managing Director     32,613
3                            Prepaid Fees         26
4                            Total Assets  $ 134,708
5        LIABILITIES AND MEMBERS' EQUITY:           
6                             LIABILITIES           
7   Accounts Payable and Accrued Expenses   $ 31,081
8                       Total Liabilities     31,081
9                         MEMBERS' EQUITY    103,627
10  Total Liabilities and Members' Equity  $ 134,708
	Saved 1022464-19.csv file to s3 bucket

File 1049
Started job with id: ed5deb8577409d483cb6ab109cc5d12dffc6a9f0e2a60de9a5d17678080074d8
Job sta

Started job with id: 54dcbd55825a7b3e397475b9ab130fdb1c542e0d25ba4797c6ae0de1e7422dc5
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                        0          1
0               Cash and cash equivalents  $ 145,958
1                        Prepaid expenses     37,660
2                          FINRA deposits      1,097
3                            Total assets  $ 184,715
4        Liabilities and members' capital           
5                            Liabilities:           
6                        Accrued expenses   $ 17,783
7                       Total liabilities     17,783
8                        Members' capital    166,932
9  Total liabilities and members' capital  $ 184,715
	Saved 1022465-09.csv file to s3 bucket

File 1056
Started job with id: 84c2a90e508a3d307e3cc3f186e946d2cb7ffc8106f9746f87744c6ce455cfcf
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRES

	Saved 1022465-14.csv file to s3 bucket

File 1061
Started job with id: b328d20fc3eb012a065729b3202bc535d51b5b664d2258d0a1acbc968bb7639c
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                         0          1
0                                   Assets           
1                                     Cash  $ 145,581
2                         Prepaid expenses     45,146
3                           FINRA deposits      4,771
4                             Total assets  $ 195,498
5         Liabilities and members' capital           
6                             Liabilities:           
7                        Due to affiliates 

In [ ]:
np.array(errorLog)

# Helper Functions

In [ ]:
def cleanNumeric(value:str) -> float:
    """
    This function converts a string to a numeric quantity, handles weird string format
    :param: value, string value with hidden numeric quanity  
    :return: floating point values
    
    Complexity -> O(n)
    
    e.g.
        In[0]: $ 19,225     ->   Out[0]: 19255
        In[0]: $ 19,225.76  ->   Out[0]: 19255.76
    """
    # perform regex operation scanning for only numeric quantities/identifiers
    cleanValue = re.sub("[^0-9|.|-]", "", value)
    if cleanValue in ['', '.', '-']:
        return 0.0
    else:
        return float(cleanValue)

In [ ]:
cleanNumeric('-$129.25') - 20